<a href="https://colab.research.google.com/github/natrask/ENM5310/blob/main/FEM_tutorial_PDEconstrainedopt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#!/usr/bin/env python
# coding: utf-8
# In[91]:
import numpy as np
import scipy
import scipy.sparse
import scipy.sparse.linalg
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.reset_default_graph()
config = tf.ConfigProto()
sess = tf.Session(config=config)

In [5]:
meshsize= 10
h = 1./float(meshsize-1)
points = np.linspace(0,1,meshsize)

Some helper functions to evaluate tent functions and their derivatives.

In [6]:
def evalPhi_i(x):
    return tf.nn.relu(1.0-(tf.abs(tf.expand_dims(x,0)-tf.expand_dims(points,1)))/h)

def evalGradPhi_i(x):
    suppPhi = tf.cast(evalPhi_i(x)>0,tf.float64)
    signPlus = tf.cast(tf.expand_dims(points,1) > tf.expand_dims(x,0),tf.float64)
    signNeg = tf.cast(tf.expand_dims(points,1) <= tf.expand_dims(x,0),tf.float64)
    return suppPhi*(-signPlus+signNeg)/h

def evalKappa(x):
    signPlus = tf.cast(tf.expand_dims(x,1) > 0.5,tf.float64)
    signNeg = tf.cast(tf.expand_dims(x,1) <= 0.5,tf.float64)
    return 1.0*signPlus + 0.1*signNeg

Build up set of quadrature points, evaluate mass and stiffness matrices. Note that we were careful to use a quadrature rule that doesn't evaluate the derivative where it is undefined.

In [27]:
# %% Get mass matrices
xql = points[:-1]+h*(0.5+1./(2.*np.sqrt(3)))
xqr = points[:-1]+h*(0.5-1./(2.*np.sqrt(3)))
xq = np.sort(np.concatenate([xql,xqr],axis=0))
quad_basisEval = evalPhi_i(xq)

# Build up parameterization of kappa as a finite element field
kappa_nodal = tf.Variable(np.ones(meshsize),dtype=tf.float64)
kappa_quadpts = tf.einsum('nq,n->q',quad_basisEval,kappa_nodal)

In [29]:

#mass matrix of P1 basis functions
nodal_basisEval = evalPhi_i(xq)
Mnodal=(h/2.)*tf.reduce_sum(tf.expand_dims(nodal_basisEval,0)*tf.expand_dims(nodal_basisEval,1),axis=2)

#stiffness matrix of P1 basis functions
nodal_gradbasisEval = evalGradPhi_i(xq)
tf.expand_dims(nodal_gradbasisEval,0)
tf.expand_dims(nodal_gradbasisEval,0)
#Snodal = (h/2.)*tf.einsum('qz,ijq->ij',evalKappa(xq),tf.expand_dims(nodal_gradbasisEval,0)*tf.expand_dims(nodal_gradbasisEval,1))
Snodal = (h/2.)*tf.einsum('q,ijq->ij',kappa_quadpts,tf.expand_dims(nodal_gradbasisEval,0)*tf.expand_dims(nodal_gradbasisEval,1))


<tf.Tensor 'Relu_29:0' shape=(10, 18) dtype=float64>

Visualize shape functions.

In [45]:
ubc_right = 1.0
ubc_left = 0.0
forcing = 0.0

projForcing = tf.einsum('ij,j->i',Mnodal,forcing*np.ones(meshsize))
solution_mat = tf.concat([tf.expand_dims(tf.one_hot(0,meshsize,dtype=tf.float64),0),
 Snodal[1:meshsize-1,:],
 tf.expand_dims(tf.one_hot(meshsize-1,meshsize,dtype=tf.float64),0)],
          axis=0)
solution_rhs = tf.concat([ubc_left*tf.ones((1,),dtype=tf.float64),
                          projForcing[1:meshsize-1],
                 ubc_right*tf.ones((1,),dtype=tf.float64)],axis=0)

u_sol = tf.linalg.solve(solution_mat,tf.expand_dims(solution_rhs,1))
dx_sol = tf.einsum('iz,iq,qz->q',u_sol,evalGradPhi_i(xq),evalKappa(xq))

In [46]:
u_target = tf.placeholder(dtype=tf.float64,shape=(meshsize,1))
LOSS_u = tf.reduce_sum((u_sol-u_target)**2)
dudx_target = tf.placeholder(dtype=tf.float64,shape=(meshsize,1))
LOSS_dudx = tf.reduce_sum((dx_sol-dudx_target)**2)

epsilon = 1.0
LOSS_total = LOSS_u + epsilon**2*LOSS_dudx

optimizer_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(CE_LOSS)
sess.run(tf.global_variables_initializer()) #initialize model